<a href="https://colab.research.google.com/github/nazarcoder123/HuggingFace_bnb_int8_T5/blob/main/HuggingFace_bnb_int8_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HuggingFace meets `bitsandbytes` for lighter models on GPU for inference

## Running T5-11b on Google Colab

 <center>
 <img src="https://s3.amazonaws.com/moonup/production/uploads/1659861207959-62441d1d9fdefb55a0b7d12c.png">
 </center>


You can run your own 8-bit model on any HuggingFace 🤗 model with just few lines of code. This notebook shows how to do it with a `T5` model that would usually require 12GB of GPU RAM.
Install the dependencies below first!


In [1]:
!pip install --quiet bitsandbytes
!pip install --quiet --upgrade transformers # Install latest version of transformers
!pip install --quiet --upgrade accelerate
!pip install --quiet sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.1 MB/s eta 0:00:00


In [2]:
!pip install transformers bitsandbytes accelerate

## Choose your model

Rerun this cell if you want to change the model!

In [3]:
model_name = "t5-3b-sharded" #@param ["t5-11b-sharded", "t5-3b-sharded"]

## Use 8bit models with `t5-3b-sharded` 🤗

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# T5-3b and T5-11B are supported!
# We need sharded weights otherwise we get CPU OOM errors
model_id=f"ybelkada/{model_name}"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_8bit = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto", load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Let's check the memory footprint of this model! 🪶

In [5]:
model_8bit.get_memory_footprint()

5300543488

For `t5-3b` the int8 model is about ~2.9GB! whereas the original model has 11GB. For `t5-11b` the int8 model is about ~11GB vs 42GB for the original model.
Now let's generate and see the qualitative results of the 8bit model!

In [8]:
max_new_tokens = 50

input_ids = tokenizer(
    "translate English to German: what is your name and where are you from and how are you why are you", return_tensors="pt"
).input_ids

outputs = model_8bit.generate(input_ids, max_new_tokens=max_new_tokens)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

was ist dein Name und woher bist du und wie bist du und warum bist du
